In [31]:
import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import warnings

Cleaning CSV Data regarding GE_15 Candidates

In [20]:
party_replace = {
    'PARTI KEADILAN RAKYAT (PKR)': 'PAKATAN HARAPAN (PH)',
    'PARTI TINDAKAN DEMOKRATIK (DAP)': 'PAKATAN HARAPAN (PH)',
    'PARTI ISLAM SE MALAYSIA (PAS)': 'PERIKATAN NASIONAL (PN)',
    'IKATAN DEMOKRATIK MALAYSIA (MUDA)': 'PAKATAN HARAPAN (PH)'
}

df = pd.read_csv('data/candidates_ge15.csv',usecols=['state','parlimen','party','result'])
df.party = df.party.replace(party_replace).fillna(df.party)
df = df[df.result == 1].drop('result',axis=1)
n_seats = df[['party']].groupby('party').size()
# df = df[~df.state.isin(['Sabah','Sarawak','W.P. Labuan'])] # to add East Msia, comment out this line
df.party = df.party.map(party_replace).fillna(df.party)
df['party_short'] = df.party.str.extract(r'\((\w+)\)')
party_abbr = dict(zip(df.party,df.party_short))
df = df.drop('party_short',axis=1)

vf = pd.read_csv('data/voters_ge15.csv',usecols=['state','parlimen','total']).rename(columns={'total':'voters'}).groupby(['state','parlimen']).sum().reset_index()

df = pd.merge(df,vf,on=['state','parlimen'],how='left')
assert len(df[df.voters.isnull()]) == 0, 'Missing data!' # ensure complete merge
n_voters = (df[['party','voters']].groupby('party').mean()).to_dict()

df = df.sort_values(by='voters',ascending=False).reset_index(drop=True)

df.to_csv("parlimen.csv")